In [1]:
import pandas as pd
import numpy as np
import spacy
import os
import csv
import sys
import json
# df = pd.read_csv('tech_file.csv')
# orig_txt = df['transcript'][25]
import torch
import random
from evaluate import load

random.seed(1234)
np.random.seed(1234)
torch.manual_seed(1234)


/home/bizon/anaconda3/envs/w210_podcast_ollama/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:

bertscore = load("bertscore")
predictions = ["hello there", "general kenobi"]
references = ["hello there", "general kenobi"]
results = bertscore.compute(predictions=predictions, references=references, lang="en")

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [3]:
results

{'precision': [0.9999999403953552, 1.0],
 'recall': [0.9999999403953552, 1.0],
 'f1': [0.9999999403953552, 1.0],
 'hashcode': 'roberta-large_L17_no-idf_version=0.3.9(hug_trans=4.31.0)'}

In [4]:
csv.field_size_limit(sys.maxsize)

podcast_data = []
row_num = 0
with open('../summarize/vtt_data.csv') as csvfile:
    reader = csv.reader(csvfile, delimiter='|')
    for row in reader:
        row_num += 1
        
        if row_num == 1:
            continue
            
        filename = row[5]
        if not filename.endswith("_large.vtt"):
            continue

        podcast = {    
            "episode_index": row[0],    
            "guest": row[1],
            "episode_name": row[2],
            "host_name": row[3],
            "episode_number": row[4],
            "transcript": row[6],
            "duration": row[7],
        }
        podcast_data.append(podcast)

In [5]:
# benchmark_summaries = {
#     "12" : "../summarize/summarized_dataset/podcast_summaries_openai_gpt35turbo_12_benchmark.json",
#     "22" : "../summarize/summarized_dataset/podcast_summaries_openai_gpt35turbo_22_benchmark.json",
#     "23" : "../summarize/summarized_dataset/podcast_summaries_openai_gpt35turbo_23_benchmark.json",
#     "94" : "../summarize/summarized_dataset/podcast_summaries_openai_gpt35turbo_94_benchmark.json",
# }

# keypoints_summaries = {
#     "12" : "../summarize/summarized_dataset/podcast_summaries_openai_gpt35turbo_12_stage3_extractkeypoints_v2b4.json",
#     "22" : "../summarize/summarized_dataset/podcast_summaries_openai_gpt35turbo_22_stage3_extractkeypoints_v2b4.json",
#     "23" : "../summarize/summarized_dataset/podcast_summaries_openai_gpt35turbo_23_stage3_extractkeypoints_v2b4.json",
#     "94" : "../summarize/summarized_dataset/podcast_summaries_openai_gpt35turbo_94_stage3_extractkeypoints_v2b4_ep94only.json",
# }

In [11]:

transcripts = []
benchmarks = []
keypoints = []

# 15 more
episodes_number = [
    3,4,5,6,7,9,10,11,13,14,15,17,18,19,20
]

count = 0
for podcast in podcast_data:

    episode_number = podcast['episode_number']
    
    if int(episode_number) not in episodes_number:
        continue
    
    transcripts.append(podcast['transcript'])
    
    print(f"Calculating score for episode: {episode_number}")    
    summary = f"../summarize/summarized_dataset_batch/podcast_summaries_openai_gpt35turbo_{episode_number}_benchmark.json"

    print("summary file path: " + str(summary))
    summary_json = None
    with open(summary) as f:
        summary_json = json.load(f)
    
    benchmarks.append(summary_json['final_summary'])
    
    # keypoints
    summary = f"../summarize/summarized_dataset_batch/podcast_summaries_openai_gpt35turbo_{episode_number}_stage3_extractkeypoints_v2b4.json"
    
    summary_json = None
    with open(summary) as f:
        summary_json = json.load(f)
    
    keypoints.append(summary_json['final_summary'])

    
benchmark_results = bertscore.compute(predictions=benchmarks, references=transcripts, lang="en")
    
print("benchmark_results:")
print(benchmark_results)

keypoint_results = bertscore.compute(predictions=keypoints, references=transcripts, lang="en")

print("keypoint_results:")
print(keypoint_results)
    

Calculating score for episode: 3
summary file path: ../summarize/summarized_dataset_batch/podcast_summaries_openai_gpt35turbo_3_benchmark.json
Calculating score for episode: 4
summary file path: ../summarize/summarized_dataset_batch/podcast_summaries_openai_gpt35turbo_4_benchmark.json
Calculating score for episode: 5
summary file path: ../summarize/summarized_dataset_batch/podcast_summaries_openai_gpt35turbo_5_benchmark.json
Calculating score for episode: 6
summary file path: ../summarize/summarized_dataset_batch/podcast_summaries_openai_gpt35turbo_6_benchmark.json
Calculating score for episode: 7
summary file path: ../summarize/summarized_dataset_batch/podcast_summaries_openai_gpt35turbo_7_benchmark.json
Calculating score for episode: 9
summary file path: ../summarize/summarized_dataset_batch/podcast_summaries_openai_gpt35turbo_9_benchmark.json
Calculating score for episode: 10
summary file path: ../summarize/summarized_dataset_batch/podcast_summaries_openai_gpt35turbo_10_benchmark.js

In [12]:

benchmark_results_avg_precision = sum(benchmark_results['precision'])/len(benchmark_results['precision'])
benchmark_results_avg_recall = sum(benchmark_results['recall'])/len(benchmark_results['recall'])

print(f"NE benchmark_results_avg_precision: {benchmark_results_avg_precision:.4f}")
print(f"NE benchmark_results_avg_recall: {benchmark_results_avg_recall:.4f}")

keypoint_results_avg_precision = sum(keypoint_results['precision'])/len(keypoint_results['precision'])
keypoint_results_avg_recall = sum(keypoint_results['recall'])/len(keypoint_results['recall'])

print(f"NE keypoint_results_avg_precision: {keypoint_results_avg_precision:.4f}")
print(f"NE keypoint_results_avg_recall: {keypoint_results_avg_recall:.4f}")

NE benchmark_results_avg_precision: 0.8276
NE benchmark_results_avg_recall: 0.8160
NE keypoint_results_avg_precision: 0.8349
NE keypoint_results_avg_recall: 0.8240
